# Make a catalog of SEDs

In [58]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
import copy
output_notebook()

Loading BokehJS ...

First we initialize an empty catalog.

In [90]:
# Make an SED catalog
cat = sed.SEDCatalog()

Now let's make a few SEDs

In [91]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [92]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = np.random.normal(loc=1.7474)*q.mas, 0.1*q.mas
    s2.make_sed()
    cat.add_SED(s2)

Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!
Make this handle asymmetric uncertainties!


Then the aggregated results look like this:

In [93]:
cat.results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0
2MASS 261832886,6000.0,4000.0,533.77,28.49,1.8734503410150887,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.185551905331586e+32,3.414062364187834e+31,-1.08,0.047,7.455,0.059,4.5,0.07,0.8896913720017506,0.0,3307.0,155.0
2MASS 261832886,6000.0,4000.0,347.16,12.05,2.8805569181126165,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,1.3475215619357513e+32,9.442019976136113e+30,-1.453,0.03,8.389,0.039,4.5,0.07,0.8896913720017506,0.0,2667.0,113.0
2MASS 261832886,6000.0,4000.0,3085.82,952.23,0.3240632795570648,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,1.0646761923068466e+34,6.571589396844378e+33,0.444,0.268,3.645,0.335,4.5,0.07,0.8896913720017506,0.0,7952.0,1264.0
2MASS 261832886,6000.0,4000.0,369.67,13.67,2.7051265261952695,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,1.5279346463339271e+32,1.1393419907785261e+31,-1.399,0.032,8.253,0.041,4.5,0.07,0.8896913720017506,0.0,2752.0,117.0
2MASS 261832886,6000.0,4000.0,502.92,25.29,1.9884004119117369,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,2.827965959492772e+32,2.8568615888860546e+31,-1.131,0.044,7.585,0.056,4.5,0.07,0.8896913720017506,0.0,3210.0,148.0
2MASS 261832886,6000.0,4000.0,554.72,30.77,1.8027266690687311,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.440519410566107e+32,3.830886677641431e+31,-1.046,0.048,7.372,0.061,4.5,0.07,0.8896913720017506,0.0,3371.0,160.0
2MASS 261832886,6000.0,4000.0,733.35,53.78,1.3636063216758894,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,6.013107338332865e+32,8.837928509481682e+31,-0.804,0.064,6.765,0.08,4.5,0.07,0.8896913720017506,0.0,3876.0,206.0


We can also plot the results by column name.

In [98]:
show(cat.plot('logg', 'Teff'))